In [3]:
import pandas

In [12]:
q_sheet_to_name = {
    "fy2019_q4": "FY 2019, 4th Q, July 1- Sept. 30, 2019",
    "fy2019_q3": "FY 2019, 3rd Q, April 1 - June 30, 2019",
    "fy2020_q1": "FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",
    "fy2020_q2_values": "FY 2020, 2nd Q, Jan. 1 - March 31 2020"
}

In [20]:
dfs = []
for sheet_name, quarter in q_sheet_to_name.items():
    df = pandas.read_excel('data/USCIS/Naturalizations by field office.xlsx', sheet_name=sheet_name, 
                           skiprows=8, usecols="A:G")
    df.columns = ['state', 'field_office', 'code', 
                  'Applications Received2', 'Approved3', 'Denied4', 'Pending5']

    df['state'] = df[df.columns[0]].fillna(axis=0, method='ffill').map(lambda name: name.strip() if type(name) == str else float('nan'))
    df = df.rename(columns={'Unnamed: 1':"field_office"})
    df['field_office'] = df.field_office.map(lambda name: name.strip() if type(name) == str else float('nan'))
    df['quarter'] = quarter
    df['year'] = df.quarter.map(lambda q: q[-4:])
    df['fy'] = df.quarter.map(lambda q: q.split(',')[0])
    df.rename(columns={'Unnamed: 2':'code'}, inplace=True)
    df['code'] = df['code'].map(lambda code: code.strip() if type(code) == str else float('nan'))
    
    dfs.append(df.dropna(subset=['field_office']))
pandas.concat(dfs).sample(6)

,state,field_office,code,Applications Received2,Approved3,Denied4,Pending5,quarter,year,fy
95,North Carolina,Charlotte,CLT,NaN,NaN,NaN,NaN,"FY 2019, 3rd Q, April 1 - June 30, 2019",2019,FY 2019
35,Florida,Kendall,KND,NaN,NaN,NaN,NaN,"FY 2019, 3rd Q, April 1 - June 30, 2019",2019,FY 2019
16,California,San Bernardino,SBD,4835,5023,562,11225,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",2019,FY 2020
46,Idaho,Boise,BOI,4080,4852,684,8272,"FY 2019, 3rd Q, April 1 - June 30, 2019",2019,FY 2019
70,Missouri,Kansas City,KAN,1269,1498,95,1683,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",2019,FY 2020
128,Washington,Seattle,SEA,4603,2539,273,18639,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",2019,FY 2020


In [21]:
new_data = pandas.concat(dfs)[['field_office',  'Applications Received2', 'Approved3', 
                    'Denied4', 'Pending5', 'quarter', 'state', 'code', 'fy']]
new_data.sample(3)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,fy
100,Columbus,4280,4346,225,16802,"FY 2019, 3rd Q, April 1 - June 30, 2019",Ohio,CLM,FY 2019
99,Columbus,1731,1665,152,2737,"FY 2019, 4th Q, July 1- Sept. 30, 2019",Ohio,CLM,FY 2019
181,Ciudad Juarez,NaN,NaN,NaN,NaN,"FY 2019, 4th Q, July 1- Sept. 30, 2019",Mexico,NaN,FY 2019


In [22]:
new_data[new_data['code'] == 'IMP']

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,fy
12,Imperial,561,390,91,935,"FY 2019, 4th Q, July 1- Sept. 30, 2019",California,IMP,FY 2019
13,Imperial,NaN,NaN,NaN,NaN,"FY 2019, 3rd Q, April 1 - June 30, 2019",California,IMP,FY 2019
12,Imperial,561,390,91,935,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",California,IMP,FY 2020
10,Imperial,477,320,115,1296,"FY 2020, 2nd Q, Jan. 1 - March 31 2020",California,IMP,FY 2020


##### def try_convert(val):
    try:
        return float(val)
    except:
        return float(0)
for col_name in ['Applications Received2', 'Approved3', 'Denied4', 'Pending5']:
    new_data[col_name] = new_data[col_name].map(lambda val: try_convert(val))

Concat in any PDFs converted to CSV

In [24]:
master_df = pandas.read_csv('data/first_pass.csv', index_col=0)

In [25]:
master_df.sample(3)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
29,Chula Vista,NaN,NaN,NaN,NaN,"FY 2018, 3rd Q, April 1-June 30, 2018",California,CVC,2018.0,FY 2018
35,Sacramento,3427.0,2507.0,264.0,4183.0,"FY 2015, 2nd Q, Jan. 1-March 31, 2015",California,SAC,2015.0,FY 2015
235,Houston,5907.0,4204.0,312.0,17379.0,"FY 2014, 4th Q, July 1-Sept. 30, 2014",Texas,HOU,2014.0,FY 2014


In [26]:
new_master = pandas.concat([master_df, new_data])

In [27]:
len(master_df), len(new_data), len(new_master)

(2639, 439, 3078)

In [28]:
new_master.to_csv('data/master_df.csv')

In [29]:
new_master[new_master['code']=='IMP']

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
29,Imperial,408,611,121,810,"FY 2013, 4th Q, July 1- Sept. 30, 2013",California,IMP,2013.0,FY 2013
29,Imperial,321,517,66,550,"FY 2014, 1st Q, Oct. 1-Dec. 31, 2013",California,IMP,2013.0,FY 2014
29,Imperial,357,337,53,546,"FY 2014, 2nd Q, Jan. 1-March 31, 2014",California,IMP,2014.0,FY 2014
29,Imperial,458,328,51,622,"FY 2014, 3rd Q, April 1-June 30, 2014",California,IMP,2014.0,FY 2014
29,Imperial,299,400,81,421,"FY 2014, 4th Q, July 1-Sept. 30, 2014",California,IMP,2014.0,FY 2014
29,Imperial,231,241,71,340,"FY 2015, 1st Q, Oct. 1-Dec. 31, 2014",California,IMP,2014.0,FY 2015
29,Imperial,314,215,42,399,"FY 2015, 2nd Q, Jan. 1-March 31, 2015",California,IMP,2015.0,FY 2015
29,Imperial,378,217,53,508,"FY 2015, 3rd Q, April 1-June 30, 2015",California,IMP,2015.0,FY 2015
29,Imperial,298,307,46,476,"FY 2015, 4th Q, July 1-Sept. 30, 2015",California,IMP,2015.0,FY 2015
29,Imperial,363,264,37,557,"FY 2016, 1st Q, Oct. 1-Dec.31, 2015",California,IMP,2015.0,FY 2016
